In [1]:
import numpy as np
np.random.seed(123)

In [2]:
import pandas as pd
import subprocess
from scipy.sparse import csr_matrix, hstack
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import KFold
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import PReLU

C:\Users\Jenny\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using Theano backend.
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.


In [3]:
train = pd.read_csv('C:\\hudsondata\\Machine Learning\\Kaggle\AllState\\train.csv')
test = pd.read_csv('C:\\hudsondata\\Machine Learning\\Kaggle\AllState\\test.csv')

In [4]:
index = list(train.index)
print index[0:10]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [5]:
np.random.shuffle(index)

In [7]:
print index[0:10]

[33898, 43407, 3458, 100139, 60086, 117290, 89806, 7751, 164059, 143297]


In [8]:
train = train.iloc[index]

In [9]:
'train = train.iloc[np.random.permutation(len(train))]'

'train = train.iloc[np.random.permutation(len(train))]'

In [10]:
test['loss'] = np.nan

In [11]:
y = np.log(train['loss'].values+200)
id_train = train['id'].values
id_test = test['id'].values

In [12]:
ntrain = train.shape[0]
tr_te = pd.concat((train,test),axis=0)

In [13]:
sparse_data = []

In [14]:
f_cat = [f for f in tr_te.columns if 'cat' in f]
for f in f_cat:
    dummy = pd.get_dummies(tr_te[f].astype('category'))
    tmp = csr_matrix(dummy)
    sparse_data.append(tmp)

In [15]:
f_num = [f for f in tr_te.columns if 'cont' in f]
scaler = StandardScaler()
tmp = csr_matrix(scaler.fit_transform(tr_te[f_num]))
sparse_data.append(tmp)

In [17]:
del tr_te, train, test

In [18]:
xtr_te = hstack(sparse_data, format = 'csr')

In [19]:
xtrain = xtr_te[:ntrain,:]
xtest = xtr_te[ntrain:,:]

In [20]:
print xtrain.shape

(188318, 1190)


In [21]:
xtest.shape

(125546, 1190)

In [32]:
def nn_model():
    model = Sequential()
    
    model.add(Dense(400, input_dim = xtrain.shape[1], init = 'he_normal'))
    model.add(PReLU())
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
    
    model.add(Dense(200, init = 'he_normal'))
    model.add(PReLU())
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    
    model.add(Dense(50, init = 'he_normal'))
    model.add(PReLU())
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    
    model.add(Dense(1, init = 'he_normal'))
    model.compile(loss='mae', optimizer = 'adadelta')
    
    return model

In [29]:
nfolds = 2
folds = KFold(len(y), n_folds = nfolds, shuffle = True, random_state = 111)

In [30]:
i = 0
nbags = 1
nepochs = 2
pred_oob = np.zeros(xtrain.shape[0])
pred_test = np.zeros(xtest.shape[0])

In [33]:
for (inTr, inTe) in folds:
    xtr = xtrain[inTr]
    ytr = y[inTr]
    xte = xtrain[inTe]
    yte = y[inTe]
    pred = np.zeros(xte.shape[0])
    for j in range(nbags):
        model = nn_model()
        fit = model.fit_generator(generator = batch_generator(xtr,ytr, 128,True), nb_epoch = nepochs, samples_per_epoch=xtr.shape[0], verbose=0)
        pred += np.exp(model.predict_generator(generator=batch_generatorp(xtest, 800, False), val_samples = xte.shape[0])[:,0])-200
        pred_test += np.exp(model.predict_generator(generator = batch_generatorp(xtest,800, False), val_samples = xtest.shape[0])[:,0])-200
    pred /= nbags
    pred_oob[inTe] = pred
    score = mean_absolute_error(np.exp(yte)-200,pred)
    i +=1
    print 'Fold ', i, '- MAE:', score

KeyboardInterrupt: 

In [27]:
def batch_generator(X, y, batch_size, shuffle):
    number_of_batches = np.ceil(X.shape[0]/batch_size)
    counter = 0
    sample_index = np.arange(X.shape[0])
    if shuffle:
        np.random.shuffle(sample_index)
    while True:
        batch_index = sample_index[batch_size*counter:batch_size*(counter+1)]
        X_batch = X[batch_index,:].toarray()
        y_batch = y[batch_index]
        counter +=1
        yield X_batch, y_batch
        if (counter==number_of_batches):
            if shuffle:
                np.random.shuffle(sample_index)
            counter =0

In [28]:
def batch_generatorp(X, batch_size, shuffle):
    number_of_batches = X.shape[0]/np.ceil(X.shape[0]/batch_size)
    counter = 0
    sample_index = np.arange(X.shape[0])
    while True:
        batch_index = sample_index[batch_size*counter:batch_size*(counter+1)]
        X_batch = X[batch_index,:].toarray()
        counter+=1
        yield X_batch
        if (counter == number_of_batches):
            counter = 0